In [5]:
import yfinance as yf
eth = yf.Ticker("ETH-USD")
df = eth.history(start="2024-07-01", end="2025-10-01")
print(df.head(), df.tail())
df.to_csv("eth_1y_yfinance.csv")


                                  Open         High          Low        Close  \
Date                                                                            
2024-07-01 00:00:00+00:00  3432.607422  3513.310791  3425.323242  3440.341553   
2024-07-02 00:00:00+00:00  3439.381348  3459.761475  3399.026123  3416.354736   
2024-07-03 00:00:00+00:00  3416.254883  3426.332275  3254.521240  3292.916748   
2024-07-04 00:00:00+00:00  3291.817871  3309.203613  3054.521973  3054.521973   
2024-07-05 00:00:00+00:00  3057.833252  3106.152588  2826.014404  2981.598633   

                                Volume  Dividends  Stock Splits  
Date                                                             
2024-07-01 00:00:00+00:00  12281551839        0.0           0.0  
2024-07-02 00:00:00+00:00   9421757718        0.0           0.0  
2024-07-03 00:00:00+00:00  16121324440        0.0           0.0  
2024-07-04 00:00:00+00:00  20252514386        0.0           0.0  
2024-07-05 00:00:00+00:00  311319426

In [6]:
import numpy as np

# Calculate daily returns (percentage change)
df['returns'] = df['Close'].pct_change()

# Calculate rolling volatility (standard deviation of returns)
df['volatility_30d'] = df['returns'].rolling(window=30).std() * np.sqrt(365)  # Annualized
df['volatility_90d'] = df['returns'].rolling(window=90).std() * np.sqrt(365)  # Annualized

# Calculate average volatility over the entire period for comparison
avg_vol_30d = df['volatility_30d'].mean()
avg_vol_90d = df['volatility_90d'].mean()

# Normalized volatility (current volatility / average volatility)
df['norm_vol_30d'] = df['volatility_30d'] / avg_vol_30d
df['norm_vol_90d'] = df['volatility_90d'] / avg_vol_90d

# Calculate price volatility range (using High-Low range)
df['price_range_30d'] = df['High'].rolling(window=30).max() - df['Low'].rolling(window=30).min()
df['price_range_90d'] = df['High'].rolling(window=90).max() - df['Low'].rolling(window=90).min()

# Percentage of price range relative to current price
df['price_range_pct_30d'] = (df['price_range_30d'] / df['Close']) * 100
df['price_range_pct_90d'] = (df['price_range_90d'] / df['Close']) * 100

print("Volatility Metrics Summary:")
print(f"Average 30-day volatility (annualized): {avg_vol_30d:.2%}")
print(f"Average 90-day volatility (annualized): {avg_vol_90d:.2%}")
print("\nLatest values:")
print(df[['Close', 'volatility_30d', 'volatility_90d', 'norm_vol_30d', 'norm_vol_90d', 
          'price_range_pct_30d', 'price_range_pct_90d']].tail(10))


Volatility Metrics Summary:
Average 30-day volatility (annualized): 72.45%
Average 90-day volatility (annualized): 73.47%

Latest values:
                                 Close  volatility_30d  volatility_90d  \
Date                                                                     
2025-09-21 00:00:00+00:00  4451.329102        0.496266        0.663411   
2025-09-22 00:00:00+00:00  4202.877441        0.529471        0.675453   
2025-09-23 00:00:00+00:00  4165.503906        0.529479        0.675161   
2025-09-24 00:00:00+00:00  4153.469238        0.441808        0.675245   
2025-09-25 00:00:00+00:00  3868.333984        0.458179        0.692042   
2025-09-26 00:00:00+00:00  4035.887939        0.484857        0.696142   
2025-09-27 00:00:00+00:00  4018.658203        0.484631        0.695267   
2025-09-28 00:00:00+00:00  4141.476562        0.486957        0.696607   
2025-09-29 00:00:00+00:00  4217.341797        0.491586        0.692508   
2025-09-30 00:00:00+00:00  4145.957520        0.

In [8]:
# Filter data after October 1, 2024
df_after_oct = df[df.index >= '2024-10-01']

# Select the volatility columns
df_filtered = df_after_oct[['Close', 'volatility_30d', 'volatility_90d', 'norm_vol_30d', 
                             'norm_vol_90d', 'price_range_pct_30d', 'price_range_pct_90d']]

print(f"Data from {df_filtered.index[0]} to {df_filtered.index[-1]}")
print(f"Total rows: {len(df_filtered)}")
print("\nFirst few rows:")
print(df_filtered.head())
print("\nLast few rows:")
print(df_filtered.tail())

# Save to CSV
df_filtered.to_csv("eth_volatility_after_oct1.csv")
print("\nSaved to eth_volatility_after_oct1.csv")

Data from 2024-10-01 00:00:00+00:00 to 2025-09-30 00:00:00+00:00
Total rows: 365

First few rows:
                                 Close  volatility_30d  volatility_90d  \
Date                                                                     
2024-10-01 00:00:00+00:00  2448.921143        0.560181        0.689089   
2024-10-02 00:00:00+00:00  2365.231689        0.548900        0.677423   
2024-10-03 00:00:00+00:00  2349.791260        0.525454        0.676048   
2024-10-04 00:00:00+00:00  2414.793945        0.533053        0.675718   
2024-10-05 00:00:00+00:00  2415.631836        0.519519        0.669880   

                           norm_vol_30d  norm_vol_90d  price_range_pct_30d  \
Date                                                                         
2024-10-01 00:00:00+00:00      0.773225      0.937943            23.569740   
2024-10-02 00:00:00+00:00      0.757653      0.922063            24.403713   
2024-10-03 00:00:00+00:00      0.725290      0.920192            24.564